In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError
import boto3

In [2]:
es = Elasticsearch("http://elasticsearch:9200")
INDEX_NAME = "course-questions"

In [3]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_row = docs_response.json()

documents_row[0]["course"]

'data-engineering-zoomcamp'

In [4]:
import hashlib

documents = []

for course in documents_row:

    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        question_hash = hashlib.md5(doc["question"].encode()).hexdigest()
        doc["id"] = question_hash
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': '2d669d12c0511996b393bff34bfbcf55'}

In [14]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            # "id": {"type": "keyword"},
            "course": {"type": "keyword"},
        }
    },
}

try:
    es.indices.get(index=INDEX_NAME)
    print(f"{INDEX_NAME} already exists")
    # es.indices.delete(index=INDEX_NAME, ignore=[400, 404])
except NotFoundError:
    response = es.indices.create(index=INDEX_NAME, body=index_settings)
    print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'}


In [15]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=INDEX_NAME, id=doc["id"], document=doc)

count_response = es.count(index=INDEX_NAME)
print(count_response)

100%|██████████| 948/948 [00:03<00:00, 261.60it/s]

{'count': 732, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [16]:
question = "How do I execute a command in a running docker container?"


def create_search_query(question):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    }
                }
            }
        },
    }
    return search_query


search_query = create_search_query(question)

response = es.search(index=INDEX_NAME, body=search_query)
response["hits"]

{'total': {'value': 861, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': '410d5719a0f86af7e51fe0338c99088e',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp',
    'id': '410d5719a0f86af7e51fe0338c99088e'}},
  {'_index': 'course-questions',
   '_id': 'a96ac0fb98da777f9dbddfd50e01c4d5',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelo